In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pylab as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
class myCallback (tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.98 and logs.get('val_accuracy')>0.95):
            print("\nReached 85% accuracy so cancelling training!")
            self.model.stop_training = True

In [3]:
def create_model():
    mobilenet_v2 = "https://www.kaggle.com/models/google/mobilenet-v3/frameworks/TensorFlow2/variations/small-100-224-feature-vector/versions/1"
    mobile_net_layers = hub.KerasLayer(mobilenet_v2, input_shape=(224,224,3))
    mobile_net_layers.trainable = False

    model = tf.keras.Sequential([
        mobile_net_layers,
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(17,activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.summary()

    train_dir = '../data/training/'
    validation_dir = '../data/validation/'

    train_datagen = ImageDataGenerator(
        rescale=1. / 255.,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )

    validation_datagen = ImageDataGenerator(
        rescale=1. / 255.
    )

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        batch_size=5,
        class_mode='categorical',
        target_size=(224, 224)
    )

    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        batch_size=5,
        class_mode='categorical',
        target_size=(224, 224)
    )

    callback = myCallback()

    history = model.fit(
        train_generator,
        epochs=100,
        validation_data=validation_generator,
        callbacks=callback)
    return model

In [4]:
model = create_model()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1024)              1529968   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 17)                17425     
                                                                 
Total params: 1,547,393
Trainable params: 17,425
Non-trainable params: 1,529,968
_________________________________________________________________
Found 1067 images belonging to 17 classes.
Found 162 images belonging to 17 classes.
Epoch 1/100
214/214 [==============================] - 34s 142ms/step - loss: 1.4648 - accuracy: 0.6082 - val_loss: 0.6771 - val_accuracy: 0.8827
Epoch 2/100
214/214 [==============================] - 15s 71ms/step - los

In [5]:
# Specify the local image path
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input

local_image_path = '../data/validation/Nasi Goreng/menueditor_item_3fcdd0b34b984b60a963a5f4c78fbc85_1564688281409332709_jpg.rf.4fb497aa024ce40e93c27f7eb7d2d930.jpg'

# Make predictions for the local image
img = image.load_img(local_image_path, target_size=(224, 224))  # Adjust target_size as needed
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

# Make predictions
predictions = model.predict(img_array)
print(predictions * 100)

predicted_class = np.argmax(predictions)

print(f"The predicted class is: {predicted_class}")

1/1 [==============================] - 0s 451ms/step
[[ 0.12534413  0.26473814  3.4306154   0.98068744  4.3005314   0.334012
   0.2667377  40.746777    0.22943023  4.1489396  30.584538    5.4733057
   0.8071917   3.473497    4.438968    0.17995077  0.21472429]]
The predicted class is: 7
